In [2]:
import pandas as pd
from fuzzywuzzy import process



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
df1 = pd.read_csv('company_ratings.txt')
df2 = pd.read_csv('top_500_market_cap_comps.csv')


In [16]:
df1

,company,rating
0,Amazon,4.0
1,Deloitte,4.1
2,Walmart,3.6
3,McDonald's,3.7
4,Target,4.0
...,...,...
1975,Brooksource,3.9
1976,eTeam,3.9
1977,Lids,3.5
1978,Jenny Craig,3.5


In [17]:
merged_dfs = []

for word in df1['company']:

    matching_rows = df2[df2['longName'].str.contains(word, case=False, na=False)]
    
    if not matching_rows.empty:
        merged_dfs.append(matching_rows)

final_data_set = pd.concat(merged_dfs)
final_data_set



/var/folders/1q/hht9gtls417cj37d4wxyl08c0000gn/T/ipykernel_1092/3786720632.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matching_rows = df2[df2['longName'].str.contains(word, case=False, na=False)]


,longName,symbol,marketCap,exchange
3,"Amazon.com, Inc.",AMZN,1811770245120,NMS
16,Walmart Inc.,WMT,488643297280,NYQ
46,McDonald's Corporation,MCD,201553330176,NYQ
165,Target Corporation,TGT,75818729472,NYQ
35,Accenture plc,ACN,234871209984,NYQ
...,...,...,...,...
65,American Express Company,AXP,158136647680,NYQ
96,The Progressive Corporation,PGR,119998210048,NYQ
124,"Automatic Data Processing, Inc.",ADP,99448389632,NMS
182,McKesson Corporation,MCK,70124568576,NYQ


In [18]:
final_data_set = final_data_set.drop_duplicates()


In [19]:
def preprocess_long_name(long_name):
    # List of strings to remove from the long name
    strings_to_remove = ['Inc.', 'Corporation', 'Company', 'plc', 'Incorporated','.com']

    # Iterate over each string and remove it from the long name
    for string in strings_to_remove:
        long_name = long_name.replace(string, '').strip()

    return long_name

# Example usage:
final_data_set['processed_long_name'] = final_data_set['longName'].apply(preprocess_long_name)
final_data_set




/var/folders/1q/hht9gtls417cj37d4wxyl08c0000gn/T/ipykernel_1092/4078863496.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_set['processed_long_name'] = final_data_set['longName'].apply(preprocess_long_name)


,longName,symbol,marketCap,exchange,processed_long_name
3,"Amazon.com, Inc.",AMZN,1811770245120,NMS,"Amazon,"
16,Walmart Inc.,WMT,488643297280,NYQ,Walmart
46,McDonald's Corporation,MCD,201553330176,NYQ,McDonald's
165,Target Corporation,TGT,75818729472,NYQ,Target
35,Accenture plc,ACN,234871209984,NYQ,Accenture
...,...,...,...,...,...
139,"Arista Networks, Inc.",ANET,86827843584,NYQ,"Arista Networks,"
365,Global Payments Inc.,GPN,33130440704,NYQ,Global Payments
298,STMicroelectronics N.V.,STM,42888081408,NYQ,STMicroelectronics N.V.
96,The Progressive Corporation,PGR,119998210048,NYQ,The Progressive


In [23]:
df1=df1.drop_duplicates()
final_data_set=final_data_set.drop_duplicates()

In [24]:

def find_similar_name(name, names_list):
    match = process.extractOne(name, names_list)
    if match[1] >= 80: 
        return match[0]
    else:
        return None

final_data_set['similar_names'] = final_data_set['longName'].apply(
    lambda x: find_similar_name(x, df1['company'].tolist())
)
final_data_set['rating'] = final_data_set['similar_names'].map(df1.set_index('company')['rating'])

final_data_set.drop(columns=['similar_names'], inplace=True)

final_data_set


,longName,symbol,marketCap,exchange,processed_long_name,rating
3,"Amazon.com, Inc.",AMZN,1811770245120,NMS,"Amazon,",4.0
16,Walmart Inc.,WMT,488643297280,NYQ,Walmart,3.6
46,McDonald's Corporation,MCD,201553330176,NYQ,McDonald's,3.7
165,Target Corporation,TGT,75818729472,NYQ,Target,4.0
35,Accenture plc,ACN,234871209984,NYQ,Accenture,4.2
...,...,...,...,...,...,...
139,"Arista Networks, Inc.",ANET,86827843584,NYQ,"Arista Networks,",4.2
365,Global Payments Inc.,GPN,33130440704,NYQ,Global Payments,3.7
298,STMicroelectronics N.V.,STM,42888081408,NYQ,STMicroelectronics N.V.,4.0
96,The Progressive Corporation,PGR,119998210048,NYQ,The Progressive,4.0


In [25]:
final_data_set.to_csv('final_data_set.csv', index=False)

In [27]:
final_data_set.to_excel('final_data_set.xlsx', index=False)